# Strands Agents と Ollama モデルを使用したローカルエージェントの構築

このノートブックでは、Strands Agent と Ollama を使用してパーソナルエージェントを作成する方法を説明します。このエージェントは、ファイル操作、Web 検索、システムコマンドなど、さまざまなローカルタスクを実行できます。

## Ollama とは？

[Ollama](https://ollama.com/) は、大規模言語モデル (LLM) をローカルマシン上で実行できるオープンソースフレームワークです。これらのモデルを操作するためのシンプルな API を提供しているため、外部サービスにデータを送信したくないプライバシー重視のアプリケーションに最適です。

Ollama の主なメリット:
- **プライバシー**: すべての処理がローカルマシン上で実行されます
- **API コストなし**: 好きなだけ無料で使用できます
- **オフライン機能**: インターネット接続なしで動作します
- **カスタマイズ**: 特定の用途に合わせて微調整できます

## エージェントの詳細

<div style="float: left; margin-right: 20px;">
    
|機能                |説明                                        |
|--------------------|---------------------------------------------------|
|基盤モデル           |Ollama mモデル - ファイル操作エージェントの作成       |
|エージェントの構造    |シングルエージェントアーキテクチャ                    |


</div>


### エージェントアーキテクチャ

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

In [ ]:
!pip install -r requirements.txt

## セットアップとインストール

このノートブックを実行する前に、以下の準備が整っていることを確認してください。

1. Ollama をインストールします。[https://ollama.com/download](https://ollama.com/download) の手順に従ってください。
2. Ollama サーバーを起動します。`ollama serve`
3. Ollama を使用してモデルをダウンロードします。`ollama pull llama3.2:1b`

詳細な手順については、[ドキュメント](https://cuddly-sniffle-lrmk2y7.pages.github.io/0.1.x/user-guide/concepts/model-providers/ollama/) を参照してください。

このノートブックでは、SageMaker Studio との互換性を確保するために、Linux ディストリビューション用の Ollama をダウンロードします。これは、Workshop Studio での AWS 主導ワークショップ中にコードを実行するために行われます。このコードをローカルで実行する場合は、現在の環境に Ollama をダウンロードする手順を調整する必要があります。

In [ ]:
# # これは Linux コンピュータで動作します
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import subprocess
subprocess.Popen(['ollama', 'serve'])

In [ ]:
!ollama pull llama3.2:1b

In [ ]:
# 必要なライブラリのインポート
import os

import requests

# Strands コンポーネントのインポート
from strands import Agent, tool
from strands.models.ollama import OllamaModel

In [ ]:
# Ollama の実行確認
try:
    response = requests.get("http://localhost:11434/api/tags")
    print("✅ Ollama is running. Available models:")
    for model in response.json().get("models", []):
        print(f"- {model['name']}")
except requests.exceptions.ConnectionError:
    print("❌ Ollama is not running. Please start Ollama before proceeding.")

## カスタムツールの定義

ツールとは、エージェントが環境とインタラクトするために使用できる関数です。以下では、パーソナルエージェントに役立つツールをいくつか定義します。

In [ ]:
# ファイル操作ツール


@tool
def file_read(file_path: str) -> str:
    """Read a file and return its content.

    Args:
        file_path (str): Path to the file to read

    Returns:
        str: Content of the file

    Raises:
        FileNotFoundError: If the file doesn't exist
    """
    try:
        with open(file_path, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"


@tool
def file_write(file_path: str, content: str) -> str:
    """Write content to a file.

    Args:
        file_path (str): The path to the file
        content (str): The content to write to the file

    Returns:
        str: A message indicating success or failure
    """
    try:
        # ディレクトリが存在しない場合は作成する
        os.makedirs(os.path.dirname(os.path.abspath(file_path)), exist_ok=True)

        with open(file_path, "w") as file:
            file.write(content)
        return f"File '{file_path}' written successfully."
    except Exception as e:
        return f"Error writing to file: {str(e)}"


@tool
def list_directory(directory_path: str = ".") -> str:
    """List files and directories in the specified path.

    Args:
        directory_path (str): Path to the directory to list

    Returns:
        str: A formatted string listing all files and directories
    """
    try:
        items = os.listdir(directory_path)
        files = []
        directories = []

        for item in items:
            full_path = os.path.join(directory_path, item)
            if os.path.isdir(full_path):
                directories.append(f"Folder: {item}/")
            else:
                files.append(f"File: {item}")

        result = f"Contents of {os.path.abspath(directory_path)}:\n"
        result += (
            "\nDirectories:\n" + "\n".join(sorted(directories))
            if directories
            else "\nNo directories found."
        )
        result += (
            "\n\nFiles:\n" + "\n".join(sorted(files)) if files else "\nNo files found."
        )

        return result
    except Exception as e:
        return f"Error listing directory: {str(e)}"

## Ollama を利用したエージェントの作成

Ollama モデルと上記で定義したツールを使ってエージェントを作成します。

注: `execute_commands`、`search_files` などのツールを追加することもできます。

In [ ]:
# エージェントのための包括的なシステムプロンプトの定義
system_prompt = """
You are a helpful personal assistant capable of performing local file actions and simple tasks for the user.

Your key capabilities:
1. Read, understand, and summarize files.
2. Create and write to files.
3. List directory contents and provide information on the files.
4. Summarize text content

When using tools:
- Always verify file paths before operations
- Be careful with system commands
- Provide clear explanations of what you're doing
- If a task cannot be completed, explain why and suggest alternatives

Always be helpful, concise, and focus on addressing the user's needs efficiently.
"""

model_id = (
    "llama3.2:1b"  # Ollama で取得した任意のモデルに変更可能
)

#### Ollama モデルを設定する
Ollama サービスが http://localhost:11434 で実行されていること、および `model_id` が上記に表示された Ollama モデルのリストに含まれていることを確認してください。

In [ ]:
ollama_model = OllamaModel(
    model_id=model_id,
    host="http://localhost:11434",
    params={
        "max_tokens": 4096,  # モデルの能力に応じて調整
        "temperature": 0.7,  # 低いほど決定論的な回答、高いほど創造的な回答
        "top_p": 0.9,  # 核サンプリングパラメータ
        "stream": True,  # ストリーミング応答の有効化
    },
)

# エージェントの作成
local_agent = Agent(
    system_prompt=system_prompt,
    model=ollama_model,
    tools=[file_read, file_write, list_directory],
)

## エージェントのテスト

いくつかのサンプルタスクを使ってエージェントをテストしてみましょう。

In [ ]:
local_agent(
    "Read the file in the path: sample_file/Amazon-com-Inc-2023-Shareholder-Letter-jp.txt and summarize it in 5 bullet points in Japanese."
)

In [ ]:
# 例 2: 現在のディレクトリ内のファイルの一覧表示
response = local_agent("Show me the files in the current directory.")

In [ ]:
# 例 3: サンプルファイルの作成
response = local_agent(
    "Create a file called 'sample.txt' with the content 'This is a test file created by my Ollama agent."
)

In [ ]:
# 例 4: 複数のファイルを読んで理解した後に README ファイルを作成する
response = local_agent("Create a readme.md for the current directory")

## まとめ

このノートブックでは、Stands と Ollama を使用してローカルパーソナルエージェントを作成しました。エージェントはファイル操作（読み取り、書き込み、追加）とテキストの要約/分析を実行できます。

これは、Ollama を用いて AI モデルをローカルで実行することの威力と、Strands のツールシステムの柔軟性を組み合わせたものです。ニーズに応じてツールを追加したり、異なる Ollama モデルを使用したりすることで、このエージェントを拡張できます。

### 次のステップ（アイデア）

- さまざまな Ollama モデルを試して、エージェントの機能にどのような影響があるかを確認する
- Web 検索、メール送信、カレンダー統合などのより複雑なツールを追加する
- エージェントが過去のインタラクションを記憶するためのメモリを実装する
- エージェントと対話するためのシンプルな UI を作成する